# FIX THE AUTO DRIVE CLASS ERROR

# Libraries

In [11]:
# %pip install keyboard
# %pip install mss
# %pip install numpy
# %pip install matplotlib
# %pip install opencv-python

import pyautogui as py
import pygetwindow as gw

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL as ImageGrab

import keyboard

import os
import math
import time
import numpy
from mss import mss
import cv2
import numpy as np
import time
import pydirectinput
import threading
from datetime import datetime

import mss.tools  # Using mss.tools for safer access

# Edge Detection

In [1]:
import cv2 # Import the OpenCV library to enable computer vision
import numpy as np # Import the NumPy scientific computing library
import matplotlib.pyplot as plt # Used for plotting and error checking
 
def binary_array(array, thresh, value=0):
  """
  Return a 2D binary array (mask) in which all pixels are either 0 or 1
     
  :param array: NumPy 2D array that we want to convert to binary values
  :param thresh: Values used for thresholding (inclusive)
  :param value: Output value when between the supplied threshold
  :return: Binary 2D array...
           number of rows x number of columns = 
           number of pixels from top to bottom x number of pixels from
             left to right 
  """
  if value == 0:
    # Create an array of ones with the same shape and type as 
    # the input 2D array.
    binary = np.ones_like(array) 
         
  else:
    # Creates an array of zeros with the same shape and type as 
    # the input 2D array.
    binary = np.zeros_like(array)  
    value = 1
 
  # If value == 0, make all values in binary equal to 0 if the 
  # corresponding value in the input array is between the threshold 
  # (inclusive). Otherwise, the value remains as 1. Therefore, the pixels 
  # with the high Sobel derivative values (i.e. sharp pixel intensity 
  # discontinuities) will have 0 in the corresponding cell of binary.
  binary[(array >= thresh[0]) & (array <= thresh[1])] = value
 
  return binary
 
def blur_gaussian(channel, ksize=3):
  """
  Implementation for Gaussian blur to reduce noise and detail in the image
     
  :param image: 2D or 3D array to be blurred
  :param ksize: Size of the small matrix (i.e. kernel) used to blur
                i.e. number of rows and number of columns
  :return: Blurred 2D image
  """
  return cv2.GaussianBlur(channel, (ksize, ksize), 0)
         
def mag_thresh(image, sobel_kernel=3, thresh=(0, 255)):
  """
  Implementation of Sobel edge detection
 
  :param image: 2D or 3D array to be blurred
  :param sobel_kernel: Size of the small matrix (i.e. kernel) 
                       i.e. number of rows and columns
  :return: Binary (black and white) 2D mask image
  """
  # Get the magnitude of the edges that are vertically aligned on the image
  sobelx = np.absolute(sobel(image, orient='x', sobel_kernel=sobel_kernel))
         
  # Get the magnitude of the edges that are horizontally aligned on the image
  sobely = np.absolute(sobel(image, orient='y', sobel_kernel=sobel_kernel))
 
  # Find areas of the image that have the strongest pixel intensity changes
  # in both the x and y directions. These have the strongest gradients and 
  # represent the strongest edges in the image (i.e. potential lane lines)
  # mag is a 2D array .. number of rows x number of columns = number of pixels
  # from top to bottom x number of pixels from left to right
  mag = np.sqrt(sobelx ** 2 + sobely ** 2)
 
  # Return a 2D array that contains 0s and 1s   
  return binary_array(mag, thresh)
 
def sobel(img_channel, orient='x', sobel_kernel=3):
  """
  Find edges that are aligned vertically and horizontally on the image
     
  :param img_channel: Channel from an image
  :param orient: Across which axis of the image are we detecting edges?
  :sobel_kernel: No. of rows and columns of the kernel (i.e. 3x3 small matrix)
  :return: Image with Sobel edge detection applied
  """
  # cv2.Sobel(input image, data type, prder of the derivative x, order of the
  # derivative y, small matrix used to calculate the derivative)
  if orient == 'x':
    # Will detect differences in pixel intensities going from 
        # left to right on the image (i.e. edges that are vertically aligned)
    sobel = cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, sobel_kernel)
  if orient == 'y':
    # Will detect differences in pixel intensities going from 
    # top to bottom on the image (i.e. edges that are horizontally aligned)
    sobel = cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, sobel_kernel)
 
  return sobel
 
def threshold(channel, thresh=(128,255), thresh_type=cv2.THRESH_BINARY):
  """
  Apply a threshold to the input channel
     
  :param channel: 2D array of the channel data of an image/video frame
  :param thresh: 2D tuple of min and max threshold values
  :param thresh_type: The technique of the threshold to apply
  :return: Two outputs are returned:
             ret: Threshold that was used
             thresholded_image: 2D thresholded data.
  """
  # If pixel intensity is greater than thresh[0], make that value
  # white (255), else set it to black (0)
  return cv2.threshold(channel, thresh[0], thresh[1], thresh_type)


# Sliding Window

In [21]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Create edge detection module
class EdgeDetection:
    def threshold(self, img, thresh=(0, 255)):
        binary = np.zeros_like(img)
        binary[(img >= thresh[0]) & (img <= thresh[1])] = 255
        return True, binary

    def blur_gaussian(self, img, ksize=3):
        return cv2.GaussianBlur(img, (ksize, ksize), 0)

    def mag_thresh(self, img, sobel_kernel=3, thresh=(0, 255)):
        sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        scale_factor = np.max(gradmag)/255
        gradmag = (gradmag/scale_factor).astype(np.uint8)
        binary_output = np.zeros_like(gradmag)
        binary_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 255
        return binary_output

edge = EdgeDetection()
filename = 'D:/Thinkin in programming/Metopen/my_game_screenshots/screenshot_4.png'
 
class Lane:
  """
  Represents a lane on a road.
  """
  def __init__(self, orig_frame):
    """
      Default constructor
         
    :param orig_frame: Original camera image (i.e. frame)
    """
    self.orig_frame = orig_frame

    # This will hold an image with the lane lines       
    self.lane_line_markings = None

    # Previous polynomial coefficients
    self.prev_left_fit = None
    self.prev_right_fit = None
 
    # This will hold the image after perspective transformation
    self.warped_frame = None
    self.transformation_matrix = None
    self.inv_transformation_matrix = None
 
    # (Width, Height) of the original video frame (or image)
    self.orig_image_size = self.orig_frame.shape[::-1][1:]
 
    width = self.orig_image_size[0]
    height = self.orig_image_size[1]
    self.width = width
    self.height = height
     
    ''' Static ROI points.
      This ROI are based on the screen dimensions of 1920 x 1080 pixels. 
      The roi must satisfy the game screen dimensions.
    
    '''
    # self.roi_points = np.float32([
    #         (634, 457),   # Top-left
    #         (1266, 457),  # Top-right
    #         (1700, 600),  # Bottom-right
    #         (200, 600)    # Bottom-left
    # ])

    # ''' for width": 1270, "height": 813 screen resolution '''
    # self.roi_points = np.float32([
    #       (419, 344),  # Top-left
    #       (837, 344),  # Top-right
    #       (1124, 452), # Bottom-right
    #       (132, 452)   # Bottom-left
    #   ])
    
    ''' for 800 x 600 screen resolution '''
    self.roi_points = np.float32([
        (266, 200),  # Top-left
        (532, 200),  # Top-right
        (700, 300),  # Bottom-right
        (100, 300)  # Bottom-left
    ])
    
    """
    Calculate ROI points based on screen dimensions
    Returns points in clockwise order: top-left, top-right, bottom-right, bottom-left
    """ 
    # # # Define points as percentages of width and height
    # self.roi_points = np.float32([
    #     (self.width * 0.25, self.height * 0.5),   # Top-left (25% from left, 50% down)
    #     (self.width * 0.75, self.height * 0.5),   # Top-right (75% from left, 50% down)
    #     (self.width * 0.85, self.height * 0.9),   # Bottom-right (85% from left, 90% down)
    #     (self.width * 0.15, self.height * 0.9)    # Bottom-left (15% from left, 90% down)
    #     ])
    
    # The desired corner locations  of the region of interest
    # after we perform perspective transformation.
    # Assume image width of 600, padding == 150.
    self.padding = int(0.25 * width) # padding from side of the image in pixels

    self.desired_roi_points = np.float32([
      [self.padding, 0], # Top-left corner
      [self.padding, self.orig_image_size[1]], # Bottom-left corner         
      [self.orig_image_size[
        0]-self.padding, self.orig_image_size[1]], # Bottom-right corner
      [self.orig_image_size[0]-self.padding, 0] # Top-right corner
    ]) 
         
    # Histogram that shows the white pixel peaks for lane line detection
    self.histogram = None
         
    # Sliding window parameters
    # no of windows
    self.no_of_windows = 10

    # window size
    self.margin = int((1/12) * width)  # Window width is +/- margin
    self.minpix = int((1/24) * width)  # Min no. of pixels to recenter window
         
    # Best fit polynomial lines for left line and right line of the lane
    self.left_fit = None
    self.right_fit = None
    self.left_lane_inds = None
    self.right_lane_inds = None
    self.ploty = None
    self.left_fitx = None
    self.right_fitx = None
    self.leftx = None
    self.rightx = None
    self.lefty = None
    self.righty = None
         
    # Pixel parameters for x and y dimensions
    self.YM_PER_PIX = 10.0 / 1000 # meters per pixel in y dimension
    self.XM_PER_PIX = 3.7 / 781 # meters per pixel in x dimension
         
    # Radii of curvature and offset
    self.left_curvem = None
    self.right_curvem = None
    self.center_offset = None
 
  def calculate_car_position(self, print_to_terminal=False):
    """
    Calculate the position of the car relative to the center
         
    :param: print_to_terminal Display data to console if True       
    :return: Offset from the center of the lane
    """
    # Assume the camera is centered in the image.
    # Get position of car in centimeters
    # In warped space (800x600), car is assumed at bottom center of ROI
    car_location = (self.desired_roi_points[2][0] + self.desired_roi_points[3][0]) / 2
 
    # Lane bottoms in warped space (green lines)
    height = self.warped_frame.shape[0]  # 600
    bottom_left = self.left_fitx[-1]  # Last point of left green line
    bottom_right = self.right_fitx[-1]  # Last point of right green line

    # center of the lane in warped space 
    center_lane = (bottom_right + bottom_left)/2

    # offset in pixels
    center_offset_px = car_location - center_lane

    # Convert to centimeters using XM_PER_PIX (adjusted for game)
    center_offset = center_offset_px * self.XM_PER_PIX * 100

    lane_width_px = bottom_right - bottom_left
    lane_width_m = lane_width_px * self.XM_PER_PIX * 100
    
    # Assume car width ~0.5m, green lines ~0.1m each
    offset_adjustment = (0.5 / lane_width_m) * (bottom_right - bottom_left) * self.XM_PER_PIX * 100 / 2
    center_offset -= offset_adjustment if center_offset > 0 else -offset_adjustment
 
    if print_to_terminal == True:
      print(str(center_offset) + 'cm')
             
    self.center_offset = center_offset
       
    return center_offset
 
  def calculate_curvature(self, print_to_terminal=False):
    """
    Calculate the road curvature in meters.
 
    :param: print_to_terminal Display data to console if True
    :return: Radii of curvature
    """
    ym_per_pix = 10/600 # meters per pixel in y dimension
    xm_per_pix = 2/800 #adjust for 800 x 600 screen resolution
    ploty = self.ploty
    left_fitx = self.left_fitx
    right_fitx = self.right_fitx

    # Define y-value where we want radius of curvature
    left_fit_cr = np.polyfit(ploty * ym_per_pix, left_fitx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, right_fitx * xm_per_pix, 2)

    y_eval = np.max(ploty)
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2 * right_fit_cr[0])
    
    self.left_curvem = left_curverad
    self.right_curvem = right_curverad

    return left_curverad, right_curverad     
         
  def calculate_histogram(self,frame=None,plot=True):
    """
    Calculate the image histogram to find peaks in white pixel count
         
    :param frame: The warped image
    :param plot: Create a plot if True
    """
    if frame is None:
      frame = self.warped_frame
             
    # Generate the histogram
    self.histogram = np.sum(frame[int(
              frame.shape[0]/2):,:], axis=0)
 
    if plot == True:
         
      # Draw both the image and the histogram
      figure, (ax1, ax2) = plt.subplots(2,1) # 2 row, 1 columns
      figure.set_size_inches(10, 5)
      ax1.imshow(frame, cmap='gray')
      ax1.set_title("Warped Binary Frame")
      ax2.plot(self.histogram)
      ax2.set_title("Histogram Peaks")
      plt.show()
             
    return self.histogram
 
  def display_curvature_offset(self, frame=None, plot=False):
    """
    Display curvature and offset statistics on the image
         
    :param: plot Display the plot if True
    :return: Image with lane lines and curvature
    """
    image_copy = None
    if frame is None:
      image_copy = self.orig_frame.copy()
    else:
      image_copy = frame
 
    cv2.putText(image_copy,'Curve Radius: '+str((
      self.left_curvem+self.right_curvem)/2)[:7]+' m', (int((
      5/600)*self.width), int((
      20/338)*self.height)), cv2.FONT_HERSHEY_SIMPLEX, (float((
      0.5/600)*self.width)),(
      255,255,255),2,cv2.LINE_AA)
    cv2.putText(image_copy,'Center Offset: '+str(
      self.center_offset)[:7]+' cm', (int((
      5/600)*self.width), int((
      40/338)*self.height)), cv2.FONT_HERSHEY_SIMPLEX, (float((
      0.5/600)*self.width)),(
      255,255,255),2,cv2.LINE_AA)
             
    if plot==True:       
      cv2.imshow("Image with Curvature and Offset", image_copy)
 
    return image_copy
     
  def get_lane_line_previous_window(self, left_fit, right_fit, plot=False):
    """
    Use the lane line from the previous sliding window to get the parameters
    for the polynomial line for filling in the lane line
    :param: left_fit Polynomial function of the left lane line
    :param: right_fit Polynomial function of the right lane line
    :param: plot To display an image or not
    """
    # margin is a sliding window parameter
    margin = self.margin
 
    # Find the x and y coordinates of all the nonzero 
    # (i.e. white) pixels in the frame.         
    nonzero = self.warped_frame.nonzero()  
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
         
    # Store left and right lane pixel indices
    left_lane_inds = ((nonzerox > (left_fit[0]*(
      nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (
      nonzerox < (left_fit[0]*(
      nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(
      nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (
      nonzerox < (right_fit[0]*(
      nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))           
    self.left_lane_inds = left_lane_inds
    self.right_lane_inds = right_lane_inds
 
    # Get the left and right lane line pixel locations  
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]  
 
    self.leftx = leftx
    self.rightx = rightx
    self.lefty = lefty
    self.righty = righty        
     
    # Fit a second order polynomial curve to each lane line
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    self.left_fit = left_fit
    self.right_fit = right_fit
         
    # Create the x and y values to plot on the image
    ploty = np.linspace(
      0, self.warped_frame.shape[0]-1, self.warped_frame.shape[0]) 
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    self.ploty = ploty
    self.left_fitx = left_fitx
    self.right_fitx = right_fitx
         
    if plot==True:
         
      # Generate images to draw on
      out_img = np.dstack((self.warped_frame, self.warped_frame, (
                           self.warped_frame)))*255
      window_img = np.zeros_like(out_img)
             
      # Add color to the left and right line pixels
      out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
      out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [
                                                                     0, 0, 255]
      # Create a polygon to show the search window area, and recast 
      # the x and y points into a usable format for cv2.fillPoly()
      margin = self.margin
      left_line_window1 = np.array([np.transpose(np.vstack([
                                    left_fitx-margin, ploty]))])
      left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([
                                    left_fitx+margin, ploty])))])
      left_line_pts = np.hstack((left_line_window1, left_line_window2))
      right_line_window1 = np.array([np.transpose(np.vstack([
                                     right_fitx-margin, ploty]))])
      right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([
                                     right_fitx+margin, ploty])))])
      right_line_pts = np.hstack((right_line_window1, right_line_window2))
             
      # Draw the lane onto the warped blank image
      cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
      cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
      result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
       
      # Plot the figures 
      figure, (ax1, ax2, ax3) = plt.subplots(3,1) # 3 rows, 1 column
      figure.set_size_inches(10, 10)
      figure.tight_layout(pad=3.0)
      ax1.imshow(cv2.cvtColor(self.orig_frame, cv2.COLOR_BGR2RGB))
      ax2.imshow(self.warped_frame, cmap='gray')
      ax3.imshow(result)
      ax3.plot(left_fitx, ploty, color='yellow')
      ax3.plot(right_fitx, ploty, color='yellow')
      ax1.set_title("Original Frame")  
  def get_lane_line_indices_sliding_windows(self, plot=False):
    histogram = self.histogram
    midpoint = int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # If we don't have previous fits, start with base points
    if self.prev_left_fit is None:
        leftx_current = leftx_base
        rightx_current = rightx_base
    histogram = self.histogram
    midpoint = int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    nwindows = 10
    window_height = int(self.warped_frame.shape[0] / nwindows)
    nonzero = self.warped_frame.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftx_current = leftx_base
    rightx_current = rightx_base
    margin = 100
    minpix = 50
    
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        win_y_low = self.warped_frame.shape[0] - (window + 1) * window_height
        win_y_high = self.warped_frame.shape[0] - window * window_height
        
        if self.prev_left_fit is not None and window > 0:
            leftx_current = int(self.prev_left_fit[0] * (win_y_high**2) + self.prev_left_fit[1] * win_y_high + self.prev_left_fit[2])
            rightx_current = int(self.prev_right_fit[0] * (win_y_high**2) + self.prev_right_fit[1] * win_y_high + self.prev_right_fit[2])
        
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))
    
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit
 
  def get_line_markings(self, frame=None):
    if frame is None:
        frame = self.orig_frame
    # Faster edge detection (adjust thresholds as needed)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    self.lane_line_markings = cv2.GaussianBlur(binary, (5, 5), 0)
    return self.lane_line_markings
         
  def histogram_peak(self):
    """
    Get the left and right peak of the histogram
 
    Return the x coordinate of the left histogram peak and the right histogram
    peak.
    """
    midpoint = int(self.histogram.shape[0]/2)
    leftx_base = np.argmax(self.histogram[:midpoint])
    rightx_base = np.argmax(self.histogram[midpoint:]) + midpoint
 
    # (x coordinate of left peak, x coordinate of right peak)
    return leftx_base, rightx_base
         
  def overlay_lane_lines(self, plot=False, left_color=(255, 0, 0),
                         right_color=(255, 0, 0), center_color=(0, 255, 0)):
    """
    Overlay detected lane lines on the original frame with custom colors and a center line.
    
    :param plot: If True, return the overlay plot (optional)
    :param left_color: BGR tuple for left lane (default green)
    :param right_color: BGR tuple for right lane (default red)
    :param center_color: BGR tuple for center line (default blue)
    :return: Frame with overlaid lanes
    """
    # Generate an image to draw the lane lines on 
    warp_zero = np.zeros_like(self.warped_frame).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))       
         
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))])

    pts_right = np.array([np.flipud(np.transpose(np.vstack([self.right_fitx, self.ploty])))])

    pts = np.hstack((pts_left, pts_right))
         
    # Draw lane on the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 255))

    # draw left and right lane lines
    for i in range(len(self.ploty)-1):
      cv2.line(color_warp, 
                (int(self.left_fitx[i]), int(self.ploty[i])), 
                (int(self.left_fitx[i+1]), int(self.ploty[i+1])), 
                left_color, 16)
      cv2.line(color_warp, 
                (int(self.right_fitx[i]), int(self.ploty[i])), 
                (int(self.right_fitx[i+1]), int(self.ploty[i+1])), 
                right_color, 16)
 
    # Draw center line
    center_x = (self.left_fitx + self.right_fitx) / 2
    for i in range(len(self.ploty)-1):
      cv2.line(color_warp, 
                (int(center_x[i]), int(self.ploty[i])), 
                (int(center_x[i+1]), int(self.ploty[i+1])), 
                center_color, 4)
      
    # Warp the blank back to original image space using inverse perspective 
    # matrix (Minv)
    # Warp back to original image space
    newwarp = cv2.warpPerspective(color_warp, self.inv_transformation_matrix, 
                                 (self.orig_frame.shape[1], self.orig_frame.shape[0]))
    
    # Combine with original image
    result = cv2.addWeighted(self.orig_frame, 1, newwarp, 0.3, 0)
    
    if plot:
        # Optional plotting code
        pass
    
    return result        
     
  def perspective_transform(self, frame=None, plot=False):
    """
    Perform the perspective transform.
    :param: frame Current frame
    :param: plot Plot the warped image if True
    :return: Bird's eye view of the current lane
    """
    if frame is None:
      frame = self.lane_line_markings
             
    # Calculate the transformation matrix
    self.transformation_matrix = cv2.getPerspectiveTransform(
      self.roi_points, self.desired_roi_points)
 
    # Calculate the inverse transformation matrix           
    self.inv_transformation_matrix = cv2.getPerspectiveTransform(
      self.desired_roi_points, self.roi_points)
 
    # Perform the transform using the transformation matrix
    self.warped_frame = cv2.warpPerspective(
      frame, self.transformation_matrix, self.orig_image_size, flags=(
     cv2.INTER_LINEAR)) 
 
    # Convert image to binary
    (thresh, binary_warped) = cv2.threshold(
      self.warped_frame, 127, 255, cv2.THRESH_BINARY)           
    self.warped_frame = binary_warped
 
    # Display the perspective transformed (i.e. warped) frame
    if plot == True:
      warped_copy = self.warped_frame.copy()
      warped_plot = cv2.polylines(warped_copy, np.int32([
                    self.desired_roi_points]), True, (147,20,255), 3)
 
      # Display the image
      while(1):
        cv2.imshow('Warped Image', warped_plot)
             
        # Press any key to stop
        if cv2.waitKey(0):
          break
 
      cv2.destroyAllWindows()   
             
    return self.warped_frame        
     
  def plot_roi(self, frame=None, plot=False):
    """
    Plot the region of interest on an image.
    :param: frame The current image frame
    :param: plot Plot the roi image if True
    """
    if plot == False:
      return
             
    if frame is None:
      frame = self.orig_frame.copy()
 
    # Overlay trapezoid on the frame
    this_image = cv2.polylines(frame, np.int32([
      self.roi_points]), True, (147,20,255), 3)
 
    # Display the image
    while(1):
      cv2.imshow('ROI Image', this_image)
             
      # Press any key to stop
      if cv2.waitKey(0):
        break
 
    cv2.destroyAllWindows()
     
def main():
     
  # Load a frame (or image)
  original_frame = cv2.imread(filename)
 
  # Create a Lane object
  lane_obj = Lane(orig_frame=original_frame)
 
  # Perform thresholding to isolate lane lines
  lane_line_markings = lane_obj.get_line_markings()
 
  # Plot the region of interest on the image
  lane_obj.plot_roi(plot=False)
 
  # Perform the perspective transform to generate a bird's eye view
  # If Plot == True, show image with new region of interest
  warped_frame = lane_obj.perspective_transform(plot=False)
 
  # Generate the image histogram to serve as a starting point
  # for finding lane line pixels
  histogram = lane_obj.calculate_histogram(plot=False)  
     
  # Find lane line pixels using the sliding window method 
  left_fit, right_fit = lane_obj.get_lane_line_indices_sliding_windows(
    plot=False)
 
  # Fill in the lane line
  lane_obj.get_lane_line_previous_window(left_fit, right_fit, plot=False)
     
  # Overlay lines on the original frame
  frame_with_lane_lines = lane_obj.overlay_lane_lines(plot=False)
 
  # Calculate lane line curvature (left and right lane lines)
  lane_obj.calculate_curvature(print_to_terminal=False)
 
  # Calculate center offset                                                                 
  lane_obj.calculate_car_position(print_to_terminal=False)
     
  # Display curvature and center offset on image
  frame_with_lane_lines2 = lane_obj.display_curvature_offset(
    frame=frame_with_lane_lines, plot=True)
     
  # Create the output file name by removing the '.jpg' part
  size = len(filename)
  new_filename = filename[:size - 4]
  new_filename = new_filename + '_thresholded.jpg'     
     
  # Save the new image in the working directory
  #cv2.imwrite(new_filename, lane_line_markings)
 
  # Display the image 
  #cv2.imshow("Image", lane_line_markings) 
     
  # Display the window until any key is pressed
  cv2.waitKey(0) 
     
  # Close all windows
  cv2.destroyAllWindows() 
     
main()

# Debugging screen res and roi coordinates

In [51]:
import cv2
import numpy as np
from mss import mss


In [52]:
class LaneRealTime(Lane):
    def __init__(self, frame_shape):
        dummy_frame = np.zeros(frame_shape, dtype=np.uint8)
        super().__init__(dummy_frame)
        
        # ROI for 800x600 (scaled from your 1270x813)
        # Wider and taller ROI for better lane detection
        '''decent one'''
        # self.roi_points = np.float32([
        #     (274, 254),  # Top-left (screenshot-based)
        #     (517, 254),  # Top-right
        #     (748, 313),  # Bottom-right
        #     (13, 313)    # Bottom-left
        # ])

        '''wider roi'''
        # # Wider ROI points with same vertical position for 800 x 600
        # self.roi_points = np.float32([
        #     (200, 254),  # Top-left (moved left)
        #     (600, 254),  # Top-right (moved right)
        #     (790, 313),  # Bottom-right (moved right)
        #     (10, 313)    # Bottom-left (moved left)
        # ])

        # # Match bottom width even closer to original for 800 x  600
        # self.desired_roi_points = np.float32([
        #     [250, 0],    # Top-left (closer to 400px)
        #     [550, 0],    # Top-right
        #     [750, 600],  # Bottom-right (closer to 780px)
        #     [50, 600]    # Bottom-left
        # ])
        
        # for 400 x 300 screen resolution
        self.roi_points = np.float32([
            (137, 127),  # Top-left (274/2, 254/2)
            (258, 127),  # Top-right (517/2, 254/2)
            (374, 156),  # Bottom-right (748/2, 313/2)
            (6, 156)     # Bottom-left (13/2, 313/2)
        ])
        self.desired_roi_points = np.float32([
            [50, 0],     # Top-left (100/2)
            [350, 0],    # Top-right (700/2)
            [350, 300],  # Bottom-right (700/2, 600/2)
            [50, 300]    # Bottom-left (100/2, 600/2)
        ])

        self.XM_PER_PIX = 2 / 800
        self.YM_PER_PIX = 10 / 600
        self.transformation_matrix = cv2.getPerspectiveTransform(self.roi_points, self.desired_roi_points)
        self.inv_transformation_matrix = cv2.getPerspectiveTransform(self.desired_roi_points, self.roi_points)
        self.roi_image = None
        self.prev_left_fit = None
        self.prev_right_fit = None
        self.debug = False  # Toggle debug

    def process_frame(self, frame):
        self.orig_frame = frame

        '''Try Grayscale''' 
        # result decent
        # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # use hsv to detect green
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Use Canny edge detection instead of HSV
        # gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        # edges = cv2.Canny(gray, 50, 150)
        # edges = cv2.dilate(edges, None, iterations=1)
        
        # edges = cv2.resize(edges, 
        #                    (frame.shape[1], frame.shape[0]), 
        #                    interpolation=cv2.INTER_NEAREST)
        
        # # Apply ROI mask
        # mask = np.zeros_like(edges)
        # cv2.fillPoly(mask, [np.int32(self.roi_points)], 255)
        # self.lane_line_markings = cv2.bitwise_and(edges, mask)
        
        # if self.debug:
        #     cv2.imshow("Edge Mask", self.lane_line_markings)

        # # black lane markings are in the range of 0-10
        # lower_black = np.array([0, 0, 0])
        # upper_black = np.array([10, 255, 255])
        # black_mask = cv2.inRange(hsv, lower_black, upper_black)

        # try green marking
        lower_green = np.array([40, 50, 50])
        upper_green = np.array([80, 255, 255])
        green_mask = cv2.inRange(hsv, lower_green, upper_green)

        # Ensure green = 255 (white), non-green = 0 (black)
        self.lane_line_markings = green_mask  # Already binary (0 or 255)
        
        # Debug: Show mask before warp
        # cv2.imshow("Green Mask", self.lane_line_markings)
        
        # Warp
        self.warped_frame = cv2.warpPerspective(
            self.lane_line_markings, 
            self.transformation_matrix, 
            (400, 300), 
            flags=cv2.INTER_LINEAR
        )

        self.calculate_histogram(plot=False)
        try:
            left_fit, right_fit = self.get_lane_line_indices_sliding_windows(plot=False)
            if self.prev_left_fit is not None:
                left_fit = 0.8 * self.prev_left_fit + 0.2 * left_fit
                right_fit = 0.8 * self.prev_right_fit + 0.2 * right_fit
            self.prev_left_fit = left_fit
            self.prev_right_fit = right_fit
            self.get_lane_line_previous_window(left_fit, right_fit, plot=False)
            self.ploty = np.linspace(0, self.warped_frame.shape[0] - 1, self.warped_frame.shape[0])
            self.left_fitx = np.clip(left_fit[0] * self.ploty**2 + left_fit[1] * self.ploty + left_fit[2], 50, 350)
            self.right_fitx = np.clip(right_fit[0] * self.ploty**2 + right_fit[1] * self.ploty + right_fit[2], 50, 350)
            self.curve_radius = self.calculate_curvature()
            self.calculate_car_position()
            result = self.overlay_lane_lines(plot=False)
            result = self.display_curvature_offset(result, plot=False)
        except Exception as e:
            print(f"Lane detection failed: {e}")
            result = frame.copy()
        self.roi_image = frame.copy()
        cv2.polylines(self.roi_image, [np.int32(self.roi_points)], isClosed=True, color=(0, 0, 255), thickness=2)
        # cv2.imshow("Warped Frame", self.warped_frame)
        return result

def real_time_lane_detection():
    sct = mss()
    monitor = {"top": 300, "left": 100, "width": 400, "height": 300}  # 800x600 window
    lane_detector = LaneRealTime((600, 800, 3))
    
    while True:
        screenshot = sct.grab(monitor)
        frame = np.array(screenshot)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)
        
        processed_frame = lane_detector.process_frame(frame)
        
        cv2.imshow("Lane Detection", processed_frame)
        cv2.imshow("ROI Image", lane_detector.roi_image)
        cv2.imshow("Warped Frame", lane_detector.warped_frame)
        # print(f"Offset: {lane_detector.center_offset:.1f} cm, Radius: {lane_detector.curve_radius:.1f} m")
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cv2.destroyAllWindows()

if __name__ == "__main__":
    real_time_lane_detection()

# Driving Algo

In [49]:
import cv2
import numpy as np
import mss
import time
import pydirectinput
import threading

class LaneRealTime(Lane):
    def __init__(self, frame_shape):
        dummy_frame = np.zeros(frame_shape, dtype=np.uint8)
        super().__init__(dummy_frame)
        # Adjusted ROI points for 400x300 frame (halved from 800x600)
        self.roi_points = np.float32([
            (137, 127),  # Top-left (274/2, 254/2)
            (258, 127),  # Top-right (517/2, 254/2)
            (374, 156),  # Bottom-right (748/2, 313/2)
            (6, 156)     # Bottom-left (13/2, 313/2)
        ])
        self.desired_roi_points = np.float32([
            [50, 0],     # Top-left (100/2)
            [350, 0],    # Top-right (700/2)
            [350, 300],  # Bottom-right (700/2, 600/2)
            [50, 300]    # Bottom-left (100/2, 600/2)
        ])
        self.transformation_matrix = cv2.getPerspectiveTransform(self.roi_points, self.desired_roi_points)
        self.inv_transformation_matrix = cv2.getPerspectiveTransform(self.desired_roi_points, self.roi_points)
        self.roi_image = None
        self.frame_count = 0
        self.skip_frames = 2
        self.processing_scale = 0.3  # Adjusted for smaller image

    def process_frame(self, frame):
        self.frame_count += 1
        if self.frame_count % self.skip_frames != 0:
            return self.roi_image.copy() if self.roi_image is not None else np.zeros((300, 400, 3), dtype=np.uint8)
        
        self.orig_frame = frame
        small_frame = cv2.resize(frame, None, 
                                 fx=self.processing_scale, 
                                 fy=self.processing_scale, 
                                 interpolation=cv2.INTER_AREA)

        hsv = cv2.cvtColor(small_frame, cv2.COLOR_BGR2HSV)
        lower_green = np.array([40, 50, 50])
        upper_green = np.array([80, 255, 255])
        green_mask = cv2.inRange(hsv, lower_green, upper_green)
        
        green_mask = cv2.resize(green_mask, 
                                (frame.shape[1], frame.shape[0]), 
                                interpolation=cv2.INTER_NEAREST)
        
        self.lane_line_markings = green_mask
        
        self.warped_frame = cv2.warpPerspective(
            self.lane_line_markings, 
            self.transformation_matrix, 
            (400, 300),  # Adjusted for new size
            flags=cv2.INTER_LINEAR
        )
        
        try:
            self.calculate_histogram(plot=False)
            left_fit, right_fit = self.get_lane_line_indices_sliding_windows(plot=False)
            self.get_lane_line_previous_window(left_fit, right_fit, plot=False)
            
            self.ploty = np.linspace(0, self.warped_frame.shape[0] - 1, self.warped_frame.shape[0])
            self.left_fitx = np.clip(left_fit[0] * self.ploty**2 + left_fit[1] * self.ploty + left_fit[2], 50, 350)  # Adjusted bounds
            self.right_fitx = np.clip(right_fit[0] * self.ploty**2 + right_fit[1] * self.ploty + right_fit[2], 50, 350)
            
            self.calculate_curvature()
            self.calculate_car_position()
            
            result = self.overlay_lane_lines(plot=False, left_color=(128, 0, 128), right_color=(128, 0, 128), center_color=(0, 0, 255))
            result = self.display_curvature_offset(result, plot=False)
        except Exception as e:
            print(f"Lane detection failed: {e}")
            result = frame.copy()
        
        self.roi_image = frame.copy()
        cv2.polylines(self.roi_image, [np.int32(self.roi_points)], isClosed=True, color=(0, 0, 255), thickness=2)
        
        return result

class AutoDriver:
    def __init__(self, screen_region):
        self.screen_region = screen_region
        self.steering_sensitivity = 0.5
        self.speed_control = 0.1
        self.center_threshold = 15
        self.is_running = False
        self.current_speed = 0
        self.current_steering = 0
        self.last_control_time = time.time()
        self.control_delay = 0.1
        self.keys = {'forward': 'w', 'left': 'a', 'right': 'd', 'brake': 'space'}
        self.debug = True
        self.drive_thread = None
        self.last_center_offset = 0
        self.last_steering = 0
        self.failure_count = 0
        self.lane_detector = LaneRealTime((300, 400, 3))  # Adjusted frame shape

    def capture_screen(self):
        try:
            with mss.mss() as sct:
                sct_img = sct.grab(self.screen_region)
                return cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGRA2BGR)
        except Exception as e:
            print(f"Screen capture error: {e}")
            return np.zeros((300, 400, 3), dtype=np.uint8)

    def control_steering(self, center_offset, curve_radius):
        raw_steering = center_offset / 100.0 * self.steering_sensitivity
        steering_value = 0.7 * self.last_steering + 0.3 * raw_steering
        self.last_steering = steering_value
        if abs(center_offset) < self.center_threshold:
            steering_value = steering_value * 0.4
        if curve_radius < 300:
            curve_factor = min(1.0, 300 / max(curve_radius, 50)) ** 2
            steering_value += curve_factor * 0.3 * (-1 if center_offset < 0 else 1)
        return max(-1.0, min(1.0, steering_value))

    def control_speed(self, curve_radius, center_offset):
        speed_value = self.speed_control
        if curve_radius < 300:
            speed_factor = min(1.0, curve_radius / 300) ** 2
            speed_value *= speed_factor
        if abs(center_offset) > 50:
            speed_value *= 0.8
        return max(0.05, speed_value)

    def apply_controls(self, steering_value, speed_value):
        if time.time() - self.last_control_time < self.control_delay:
            return
        self.last_control_time = time.time()
        self.reset_controls()
        press_duration = min(0.05, abs(steering_value) * 0.2)
        if steering_value < -0.1:
            pydirectinput.keyDown(self.keys['left'])
            time.sleep(press_duration)
            pydirectinput.keyUp(self.keys['left'])
            self.current_steering = -1
        elif steering_value > 0.1:
            pydirectinput.keyDown(self.keys['right'])
            time.sleep(press_duration)
            pydirectinput.keyUp(self.keys['right'])
            self.current_steering = 1
        else:
            self.current_steering = 0
        if speed_value > 0.05:
            pydirectinput.keyDown(self.keys['forward'])
            self.current_speed = speed_value
        elif self.failure_count > 5:
            pydirectinput.keyDown(self.keys['brake'])
            time.sleep(0.1)
            pydirectinput.keyUp(self.keys['brake'])
            self.current_speed = 0

    def reset_controls(self):
        for key in self.keys.values():
            pydirectinput.keyUp(key)

    def drive_loop(self):
        frame_count = 0
        while self.is_running:
            start_time = time.time()
            frame = self.capture_screen()
            processed_frame = self.lane_detector.process_frame(frame)
            
            try:
                center_offset = self.lane_detector.calculate_car_position()
                curve_radius = (self.lane_detector.left_curvem + self.lane_detector.right_curvem) / 2
                self.last_center_offset = center_offset
                self.failure_count = 0
            except Exception as e:
                print(f"Calculation error: {e}")
                center_offset = self.last_center_offset
                curve_radius = 500
                self.failure_count += 1

            steering_value = self.control_steering(center_offset, curve_radius)
            speed_value = self.control_speed(curve_radius, center_offset)
            self.apply_controls(steering_value, speed_value)

            if self.debug:
                cv2.putText(processed_frame, f"Steering: {steering_value:.2f}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(processed_frame, f"Speed: {speed_value:.2f}", (10, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(processed_frame, f"Offset: {center_offset:.1f} cm", (10, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(processed_frame, f"Curve: {curve_radius:.1f} m", (10, 120), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.imshow("Lane Detection Driver", processed_frame)
                if self.lane_detector.roi_image is not None and self.lane_detector.roi_image.size > 0:
                    cv2.imshow("ROI Image", self.lane_detector.roi_image)
                if frame_count % 10 == 0:
                    fps = 1.0 / (time.time() - start_time)
                    print(f"FPS: {fps:.2f}, Steering: {steering_value:.2f}, Speed: {speed_value:.2f}")
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.stop()
                break
            frame_count += 1

    def start(self):
        if not self.is_running:
            self.is_running = True
            self.drive_thread = threading.Thread(target=self.drive_loop)
            self.drive_thread.daemon = True
            self.drive_thread.start()
            print("Autonomous driving started")

    def stop(self):
        if self.is_running:
            self.is_running = False
            self.reset_controls()
            if self.drive_thread:
                self.drive_thread.join(timeout=1.0)
            cv2.destroyAllWindows()
            print("Autonomous driving stopped")

def main():
    # Adjusted screen region for smaller capture
    screen_region = {"top": 300, "left": 100, "width": 400, "height": 300}
    driver = AutoDriver(screen_region)
    print("Starting in 3 seconds...")
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    driver.start()
    try:
        while driver.is_running:
            time.sleep(0.1)
    except KeyboardInterrupt:
        driver.stop()

if __name__ == "__main__":
    main()

Starting in 3 seconds...
3...
2...
1...
Autonomous driving started
Calculation error: 'NoneType' object has no attribute 'shape'
FPS: 1.93, Steering: 0.00, Speed: 0.10
FPS: 1.94, Steering: -0.07, Speed: 0.10
FPS: 1.50, Steering: -0.30, Speed: 0.05
FPS: 1.93, Steering: -0.10, Speed: 0.10
FPS: 1.31, Steering: 0.45, Speed: 0.06
FPS: 1.94, Steering: -0.02, Speed: 0.10
FPS: 1.34, Steering: -0.11, Speed: 0.10
FPS: 1.95, Steering: 0.01, Speed: 0.10
FPS: 1.93, Steering: -0.01, Speed: 0.10
FPS: 1.95, Steering: 0.03, Speed: 0.10
FPS: 1.51, Steering: 0.42, Speed: 0.05
FPS: 1.94, Steering: -0.00, Speed: 0.10
FPS: 1.35, Steering: 0.12, Speed: 0.10


C:\Users\acer\AppData\Local\Temp\ipykernel_52940\3880499170.py:176: RuntimeWarning: divide by zero encountered in scalar divide
  offset_adjustment = (0.5 / lane_width_m) * (bottom_right - bottom_left) * self.XM_PER_PIX * 100 / 2
C:\Users\acer\AppData\Local\Temp\ipykernel_52940\3880499170.py:176: RuntimeWarning: invalid value encountered in scalar multiply
  offset_adjustment = (0.5 / lane_width_m) * (bottom_right - bottom_left) * self.XM_PER_PIX * 100 / 2


FPS: 1.51, Steering: 1.00, Speed: 0.05
FPS: 1.51, Steering: 1.00, Speed: 0.05
Autonomous driving stopped


# Try Optimize the FPS

In [17]:
import cv2
import numpy as np
import mss
import time
import pydirectinput
import threading
from queue import Queue
from concurrent.futures import ThreadPoolExecutor

class LaneRealTime(Lane):
    def __init__(self, frame_shape):
        dummy_frame = np.zeros(frame_shape, dtype=np.uint8)
        super().__init__(dummy_frame)
        self.roi_points = np.float32([
            (200, 254),  # Top-left
            (600, 254),  # Top-right
            (790, 313),  # Bottom-right
            (10, 313)    # Bottom-left
        ])
        self.desired_roi_points = np.float32([
            [250, 0],    # Top-left
            [550, 0],    # Top-right
            [750, 600],  # Bottom-right
            [50, 600]    # Bottom-left
        ])
        self.XM_PER_PIX = 2 / 800
        self.YM_PER_PIX = 10 / 600
        self.transformation_matrix = cv2.getPerspectiveTransform(self.roi_points, self.desired_roi_points)
        self.inv_transformation_matrix = cv2.getPerspectiveTransform(self.desired_roi_points, self.roi_points)
        self.roi_image = np.zeros((600, 800, 3), dtype=np.uint8)  # Default valid image
        self.prev_left_fit = None
        self.prev_right_fit = None
        self.debug = True  # Keep debug on for now to monitor
        self.processing_scale = 0.1
        self.skip_frames = 4
        self.frame_count = 0
        self.ploty = None
        self.left_fitx = None
        self.right_fitx = None
        self.warped_frame = None

    def process_frame(self, frame):
        if frame is None or frame.size == 0 or frame.shape[0] == 0 or frame.shape[1] == 0:
            print("Invalid input frame in process_frame")
            return self.roi_image.copy()  # Return valid default

        self.frame_count += 1
        if self.frame_count % self.skip_frames != 0:
            return self.roi_image.copy() if self.roi_image is not None else np.zeros((600, 800, 3), dtype=np.uint8)
        
        self.orig_frame = frame
        small_frame = cv2.resize(frame, None, 
                                 fx=self.processing_scale, 
                                 fy=self.processing_scale, 
                                 interpolation=cv2.INTER_AREA)
        
        # hsv = cv2.cvtColor(small_frame, cv2.COLOR_BGR2HSV)
        # lower_green = np.array([40, 50, 50])
        # upper_green = np.array([80, 255, 255])
        # green_mask = cv2.inRange(hsv, lower_green, upper_green)
        
        # green_mask = cv2.resize(green_mask, 
        #                         (frame.shape[1], frame.shape[0]), 
        #                         interpolation=cv2.INTER_NEAREST)
        
        # self.lane_line_markings = green_mask

        
        # Use Canny edge detection instead of HSV
        gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        edges = cv2.dilate(edges, None, iterations=1)
        
        edges = cv2.resize(edges, 
                           (frame.shape[1], frame.shape[0]), 
                           interpolation=cv2.INTER_NEAREST)
        
        # Apply ROI mask
        mask = np.zeros_like(edges)
        cv2.fillPoly(mask, [np.int32(self.roi_points)], 255)
        self.lane_line_markings = cv2.bitwise_and(edges, mask)
        
        if self.debug:
            cv2.imshow("Edge Mask", self.lane_line_markings)
        
        self.warped_frame = cv2.warpPerspective(
            self.lane_line_markings, 
            self.transformation_matrix, 
            (800, 600), 
            flags=cv2.INTER_LINEAR | cv2.WARP_FILL_OUTLIERS
        )
        if self.debug:
            cv2.imshow("Warped Frame", self.warped_frame)
        
        self.calculate_histogram(plot=False)
        try:
            left_fit, right_fit = self.get_lane_line_indices_sliding_windows(plot=False)
            if left_fit is not None and right_fit is not None:
                if self.prev_left_fit is not None:
                    left_fit = 0.9 * self.prev_left_fit + 0.1 * left_fit
                    right_fit = 0.9 * self.prev_right_fit + 0.1 * right_fit
                self.prev_left_fit = left_fit
                self.prev_right_fit = right_fit
                self.get_lane_line_previous_window(left_fit, right_fit, plot=False)
                self.ploty = np.linspace(0, self.warped_frame.shape[0] - 1, self.warped_frame.shape[0])
                self.left_fitx = np.clip(left_fit[0] * self.ploty**2 + left_fit[1] * self.ploty + left_fit[2], 50, 750)
                self.right_fitx = np.clip(right_fit[0] * self.ploty**2 + right_fit[1] * self.ploty + right_fit[2], 50, 750)
                self.calculate_curvature()
                self.calculate_car_position()
            else:
                self.ploty = np.linspace(0, self.warped_frame.shape[0] - 1, self.warped_frame.shape[0]) if self.ploty is None else self.ploty
                self.left_fitx = np.zeros_like(self.ploty) if self.left_fitx is None else self.left_fitx
                self.right_fitx = np.zeros_like(self.ploty) if self.right_fitx is None else self.right_fitx
            result = self.overlay_lane_lines(plot=False)
            result = self.display_curvature_offset(result, plot=False)
        except Exception as e:
            print(f"Lane detection failed: {e}")
            result = self.roi_image.copy()
        
        if result is None or result.size == 0 or result.shape[0] == 0 or result.shape[1] == 0:
            print("Invalid processed frame, returning default")
            result = self.roi_image.copy()
        
        self.roi_image = frame.copy()
        cv2.polylines(self.roi_image, [np.int32(self.roi_points)], isClosed=True, color=(0, 0, 255), thickness=2)
        return result
    
class AutoDriver:
    def __init__(self, screen_region):
        self.screen_region = screen_region
        self.is_running = False
        self.speed_control = 0.2  # Increased for testing
        self.steering_sensitivity = 0.5
        self.center_threshold = 10
        self.last_steering = 0
        self.lane_detector = LaneRealTime((600, 800, 3))
        self.drive_thread = None
        self.debug = True
        self.failure_count = 0
        self.last_center_offset = 0
        self.last_curve_radius = 10000
        self.last_control_time = 0
        self.control_delay = 0.05
        self.current_steering = 0
        self.current_speed = 0

        self.frame_queue = Queue(maxsize=1)
        self.executor = ThreadPoolExecutor(max_workers=1)
        self.capture_thread = None
        self.process_thread = None
        
        self.keys = {
            'left': 'a',
            'right': 'd',
            'forward': 'w',
            'brake': 's'
        }

    def capture_screen(self):
        with mss.mss() as sct:
            while self.is_running:
                try:
                    if not self.frame_queue.full():
                        sct_img = sct.grab(self.screen_region)
                        frame = cv2.cvtColor(np.array(sct_img), cv2.COLOR_BGRA2BGR)
                        if frame is None or frame.size == 0 or frame.shape[0] == 0 or frame.shape[1] == 0:
                            frame = np.zeros((600, 800, 3), dtype=np.uint8)
                        self.frame_queue.put(frame, block=False)
                    else:
                        time.sleep(0.005)
                except Exception as e:
                    print(f"Screen capture error: {e}")
                    self.frame_queue.put(np.zeros((600, 800, 3), dtype=np.uint8), block=False)
                time.sleep(0.005)

    def control_steering(self, center_offset, curve_radius, lane):
        if lane.ploty is None or lane.left_fitx is None or lane.right_fitx is None:
            print("Warning: Lane attributes not initialized, using defaults")
            return self.last_steering
        
        if abs(center_offset - self.last_center_offset) > 50 or curve_radius > 5000:
            center_offset = self.last_center_offset
            curve_radius = self.last_curve_radius
        
        print(f"Offset: {center_offset:.1f} cm, Radius: {curve_radius:.1f} m")
        
        ploty = lane.ploty
        left_fitx = lane.left_fitx
        right_fitx = lane.right_fitx
        left_slope = (left_fitx[-1] - left_fitx[0]) / (ploty[-1] - ploty[0]) if ploty[-1] != ploty[0] else 0
        right_slope = (right_fitx[-1] - right_fitx[0]) / (ploty[-1] - ploty[0]) if ploty[-1] != ploty[0] else 0
        # Add threshold to reduce noise
        slope_diff = abs(left_slope - right_slope)
        curve_direction = 1 if left_slope > right_slope else -1 if slope_diff > 0.05 else 0  # Threshold for noise
        print(f"Left Slope: {left_slope:.3f}, Right Slope: {right_slope:.3f}, Direction: {curve_direction}")
        
        raw_steering = center_offset / 100.0 * self.steering_sensitivity
        steering_value = 0.7 * self.last_steering + 0.3 * raw_steering
        
        if curve_radius < 1000 and curve_direction != 0:
            curve_factor = min(1.5, 1000 / max(curve_radius, 50))
            steering_adjustment = curve_factor * 0.4 * curve_direction * (-1 if center_offset < 0 else 1)
            steering_value += steering_adjustment
            print(f"Curve Factor: {curve_factor:.2f}, Adjustment: {steering_adjustment:.2f}")
        if abs(center_offset) < self.center_threshold:
            steering_value = steering_value * 0.4
        
        steering_value = max(-1.0, min(1.0, steering_value))
        self.last_steering = steering_value
        self.last_center_offset = center_offset
        self.last_curve_radius = curve_radius
        print(f"Steering Value: {steering_value:.2f}")
        return steering_value

    def control_speed(self, curve_radius, center_offset):
        speed_value = self.speed_control
        if curve_radius < 500:
            speed_factor = max(0.5, curve_radius / 500)
            speed_value *= speed_factor
        # Removed offset throttle to match first pipeline
        speed_value = max(0.05, speed_value)
        print(f"Adjusted Speed: {speed_value:.2f}")
        return speed_value

    def apply_controls(self, steering_value, speed_value):
        if time.time() - self.last_control_time < self.control_delay:
            return
        self.last_control_time = time.time()
        self.reset_controls()
        press_duration = min(0.05, abs(steering_value) * 0.1)
        if steering_value < -0.2:
            pydirectinput.keyDown(self.keys['left'])
            time.sleep(press_duration)
            pydirectinput.keyUp(self.keys['left'])
            self.current_steering = -1
        elif steering_value > 0.2:
            pydirectinput.keyDown(self.keys['right'])
            time.sleep(press_duration)
            pydirectinput.keyUp(self.keys['right'])
            self.current_steering = 1
        else:
            self.current_steering = 0
        if speed_value > 0.05:
            pydirectinput.keyDown(self.keys['forward'])
            self.current_speed = speed_value
        elif self.failure_count > 5:
            pydirectinput.keyDown(self.keys['brake'])
            time.sleep(0.1)
            pydirectinput.keyUp(self.keys['brake'])
            self.current_speed = 0

    def reset_controls(self):
        for key in self.keys.values():
            pydirectinput.keyUp(key)

    def drive_loop(self):
        frame_count = 0
        while self.is_running:
            start_time = time.time()
            try:
                frame = self.frame_queue.get(timeout=0.1)
                if frame is None or frame.size == 0 or frame.shape[0] == 0 or frame.shape[1] == 0:
                    print("Invalid frame from queue, skipping")
                    continue
            except Queue.Empty:
                frame = np.zeros((600, 800, 3), dtype=np.uint8)
            
            processed_frame = self.lane_detector.process_frame(frame)
            
            if processed_frame is None or processed_frame.size == 0 or processed_frame.shape[0] == 0 or processed_frame.shape[1] == 0:
                print("Invalid processed frame, using default")
                processed_frame = np.zeros((600, 800, 3), dtype=np.uint8)

            try:
                center_offset = self.lane_detector.calculate_car_position()
                curve_radius = (self.lane_detector.left_curvem + self.lane_detector.right_curvem) / 2
                self.failure_count = 0
            except Exception as e:
                print(f"Calculation error: {e}")
                center_offset = self.last_center_offset
                curve_radius = 500
                self.failure_count += 1

            steering_value = self.control_steering(center_offset, curve_radius, self.lane_detector)
            speed_value = self.control_speed(curve_radius, center_offset)
            self.apply_controls(steering_value, speed_value)

            if self.debug:
                cv2.putText(processed_frame, f"Steering: {steering_value:.2f}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(processed_frame, f"Speed: {speed_value:.2f}", (10, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(processed_frame, f"Offset: {center_offset:.1f} cm", (10, 90), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(processed_frame, f"Curve: {curve_radius:.1f} m", (10, 120), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.imshow("Lane Detection Driver", processed_frame)
                # Removed ROI Image display to reduce load
                if frame_count % 10 == 0:
                    fps = 1.0 / (time.time() - start_time)
                    print(f"FPS: {fps:.2f}, Steering: {steering_value:.2f}, Speed: {speed_value:.2f}")
            if cv2.waitKey(1) & 0xFF == ord('q'):
                self.stop()
                break
            frame_count += 1

    def start(self):
        if not self.is_running:
            self.is_running = True
            self.capture_thread = threading.Thread(target=self.capture_screen)
            self.capture_thread.daemon = True
            self.capture_thread.start()
            self.drive_thread = threading.Thread(target=self.drive_loop)
            self.drive_thread.daemon = True
            self.drive_thread.start()
            print("Autonomous driving started")

    def stop(self):
        if self.is_running:
            self.is_running = False
            self.reset_controls()
            if self.capture_thread:
                self.capture_thread.join(timeout=1.0)
            if self.drive_thread:
                self.drive_thread.join(timeout=1.0)
            cv2.destroyAllWindows()
            print("Autonomous driving stopped")

def main():
    screen_region = {"top": 300, "left": 50, "width": 800, "height": 600}
    driver = AutoDriver(screen_region)
    print("Starting in 3 seconds...")
    for i in range(3, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    driver.start()
    
    try:
        while True:
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            time.sleep(0.1)
    finally:
        driver.stop()

if __name__ == "__main__":
    main()

Starting in 3 seconds...
3...
2...
1...
Autonomous driving started
Calculation error: 'NoneType' object has no attribute 'shape'
Adjusted Speed: 0.20
FPS: 1.83, Steering: 0.00, Speed: 0.20
Calculation error: 'NoneType' object has no attribute 'shape'
Adjusted Speed: 0.20
Calculation error: 'NoneType' object has no attribute 'shape'
Adjusted Speed: 0.20
Offset: 0.3 cm, Radius: 287.3 m
Left Slope: -0.335, Right Slope: 0.322, Direction: -1
Curve Factor: 1.50, Adjustment: -0.60
Steering Value: -0.24
Adjusted Speed: 0.11
Offset: 0.3 cm, Radius: 287.3 m
Left Slope: -0.335, Right Slope: 0.322, Direction: -1
Curve Factor: 1.50, Adjustment: -0.60
Steering Value: -0.31
Adjusted Speed: 0.11
Offset: 0.3 cm, Radius: 287.3 m
Left Slope: -0.335, Right Slope: 0.322, Direction: -1
Curve Factor: 1.50, Adjustment: -0.60
Steering Value: -0.33
Adjusted Speed: 0.11
Offset: 0.3 cm, Radius: 287.3 m
Left Slope: -0.335, Right Slope: 0.322, Direction: -1
Curve Factor: 1.50, Adjustment: -0.60
Steering Value: -0.3

Exception in thread Thread-26 (drive_loop):
Traceback (most recent call last):
  File "d:\Thinkin in programming\.conda\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\acer\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "d:\Thinkin in programming\.conda\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_52940\2062082865.py", line 303, in drive_loop
  File "C:\Users\acer\AppData\Local\Temp\ipykernel_52940\2062082865.py", line 325, in stop
  File "d:\Thinkin in programming\.conda\Lib\threading.py", line 1116, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


KeyboardInterrupt: 

In [ ]:
# bottom left corner Point(x=58, y=374)
# top left corner Point(x=278, y=309)

# top right corner Point(x=632, y=309)
# bottom right corner Point(x=852, y=374)

# translate to roi points
# top left corner (278, 309)
# top right corner (632, 309)
# bottom right corner (852, 374)
# bottom left corner (58, 374)